## 

## Land - water

## Grids

In [31]:
pixel_size = 16*0.500085924269#0.000089830415712
p = 0
i = 0
minx = -175.0002673
maxx = 180
while p<(abs(minx)+maxx) and p+minx < maxx-pixel_size:
    i += 1
    p += pixel_size
print(i, p+minx)

p = 0
i = 0
miny = -56.0007209
maxy = 75.02179125847826
while p<(abs(miny)+maxy) and p+miny < maxy-pixel_size:
    i += 1
    p += pixel_size
print(i, p+miny)

44 177.06022338537613
16 72.02127571286402


Reference terrain (steep parts) + sum km^2 for water

#### Terrain and unused lands

### Macro areas

## Urban areas

## 1- Queries

Query for all the areas contains. For the areas intersecting but not contained, we arbitrarily choose one tile.

drop some artifacts (vertical and horizontal lines)

DELETE FROM urban_areas
WHERE (abs(ST_YMax(geom)-ST_YMin(geom)) >= 0.0005 AND abs(ST_XMax(geom)-ST_XMin(geom)) <= 9.08304157104235e-05)
OR (abs(ST_XMax(geom)-ST_XMin(geom)) >= 0.0005 AND abs(ST_YMax(geom)-ST_YMin(geom)) <= 9.08304157104235e-05);

### Tiles > upper-tiles > global

### tiles upgrades

In [ ]:
INSERT INTO tiles_upgrade_areas (gid, geom, tileid)
        SELECT gid, geom, tileid
        FROM (
                SELECT g.gid, g.geom, globaltile.tileid
                FROM urban_areas_view g
                INNER JOIN tiles_boundaries b ON ST_INTERSECTS(b.geom, g.geom)
                INNER JOIN tiles_land globaltile ON ST_Contains(globaltile.geom, b.geom)
                WHERE b.parent_type = 'tileglobal' AND g.type='tile8' and globaltile.type = b.parent_type
        ) tilestable;

In [ ]:
INSERT INTO tiles_upgrade_areas_tounion (gid, geom, tileid, has_neighbours)
        SELECT g.gid, g.geom, g.tileid,  EXISTS (
                    SELECT * from tiles_upgrade_areas g2 WHERE g.gid <> g2.gid AND g.tileid = g2.tileid
                    AND ST_Dwithin(g.geom, g2.geom, 0.00001)) as has_neighbours
        FROM tiles_upgrade_areas g;

#### global

In [ ]:
INSERT INTO urban_areas_tiles (gid, tileid, type)
        SELECT g.gid, 247257, 'tileglobal'
        FROM urban_areas_view g
        WHERE g.type = 'tile8' AND
        (NOT EXISTS (SELECT * FROM tiles_areas_global_from_8x8_tounion b WHERE g.gid = b.gid AND has_neighbours = 't'));


        WITH ins1 AS (
            INSERT INTO urban_areas (geom, urban_area_km2)
            SELECT geom, urban_area_km2
            FROM buffer_upgrade
            RETURNING gid, geom, urban_area_km2
           )
        INSERT INTO urban_areas_tiles (gid, tileid, type)
        SELECT gid, tileid, 'tileglobal'
        FROM ins1
        INNER JOIN buffer_upgrade t ON ins1.urban_area_km2 = t.urban_area_km2 AND st_DWithin(ins1.geom, t.geom,0) AND ST_CENTROID(ins1.geom) = ST_centroid(t.geom);

In [ ]:
INSERT INTO urban_areas_tiles (gid, tileid, type) 
SELECT g.gid, 247257, 'tileglobal'
FROM urban_areas_view g
WHERE g.type = 'tile8' AND 
(NOT EXISTS (SELECT * FROM tiles_areas_global_from_8x8_tounion b WHERE g.gid = b.gid AND has_neighbours = 't'));

### Macro

## Output CSV

### Summaries

#### CSV

#### SHP

### All urban areas per tile, per macro

#### CSV

#### Upload

### Macros

#### CSV

#### SHP

upload

## 4 Maps